
# 目标

通过实战

1. 理解EFK工作过程
2. 基于Serilog构建一个简单的Asp.Net Core 程序，并在Kibana中可视化分析

本文主体参照多篇文章并局部调整而成，网上文章大部分行不通

## 前置条件

1. Kubernete 已安装，并且已启动，此处使用Minikube
2. Kubectl 已安装
3. Helm 已安装


## 工具说明
    强烈建议使用VS Code，并安装Remote for VS Code(如果操作远程服务器的话), Docker for VS Code, K8s for VS Code

    另外，VS Code 的Terminal非常好用

## 前置软件安装

本文基于Debian

### 安装kubectl
参考： https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/#install-using-native-package-management

#### 程序安装：

```bash
sudo apt-get update
sudo apt-get install -y apt-transport-https ca-certificates curl
sudo curl -fsSLo /usr/share/keyrings/kubernetes-archive-keyring.gpg https://packages.cloud.google.com/apt/doc/apt-key.gpg
echo "deb [signed-by=/usr/share/keyrings/kubernetes-archive-keyring.gpg] https://apt.kubernetes.io/ kubernetes-xenial main" | sudo tee /etc/apt/sources.list.d/kubernetes.list
sudo apt-get update
sudo apt-get install -y kubectl
```

#### bash 自动完成配置：

```bash

echo 'source <(kubectl completion bash)' >>~/.bashrc

```

#### bash 短命令配置：

```bash

echo 'alias k=kubectl' >>~/.bashrc
echo 'complete -F __start_kubectl k' >>~/.bashrc

```

### 安装Minikube

参考：https://minikube.sigs.k8s.io/docs/start/

#### 安装

```bash

curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube_latest_amd64.deb
sudo dpkg -i minikube_latest_amd64.deb

```

#### 启动

```bash

minikube start

```

#### 基本使用
```bash

kubectl get po -A
minikube kubectl -- get po -A

minikube dashboard

```

### 安装Helm

参考：https://helm.sh/docs/intro/install/

```bash

curl https://baltocdn.com/helm/signing.asc | sudo apt-key add -
sudo apt-get install apt-transport-https --yes
echo "deb https://baltocdn.com/helm/stable/debian/ all main" | sudo tee /etc/apt/sources.list.d/helm-stable-debian.list
sudo apt-get update
sudo apt-get install helm

```

## 配置过程




### 配置 Elasticsearch

#### 调整Minikube，启用必要插件

参考 https://github.com/elastic/helm-charts/tree/master/elasticsearch/examples/minikube

TODO 不确定是否必须

```bash
#minikube addons enable default-storageclass
#minikube addons enable storage-provisioner
#minikube addons enable ingress
```

#### 安装Elasticsearch
生成修订文件
```yaml
#elasticsearch_values.yaml
---
# Permit co-located instances for solitary minikube virtual machines.
antiAffinity: "soft"
```

```bash
helm repo add elastic https://helm.elastic.co
helm repo update
helm install elasticsearch elastic/elasticsearch -f elasticsearch_values.yaml
```

#### 自测Elasticsearch
终端1
```bash
kubectl port-forward svc/elasticsearch-master 9200
```
终端2
```bash
curl localhost:9200/_cat/indices
```

### 配置Kibana
#### 安装Kibana

```bash
helm install kibana elastic/kibana 
```

#### 自测Kibana

终端1
```bash
kubectl port-forward deployment/kibana-kibana 5601 
```

终端1
```bash
open https://localhost:5601
```

### 配置Counter Pod

Add counter.yaml
```yaml
## counter.yaml
apiVersion: v1
kind: Pod
metadata:
  name: counter
spec:
  containers:
  - name: count
    image: busybox
    args: [/bin/sh, -c, 'i=0; while true; do echo "Demo log $i: $(date)"; i=$((i+1)); sleep 1; done']
```

启动 Pod

```bash
kubectl apply -f counter.yaml
```

### 配置Fluentd
基于官方调整 https://github.com/fluent/fluentd-kubernetes-daemonset/blob/master/fluentd-daemonset-elasticsearch-rbac.yaml

#### 扩展Fluentd配置以支持解析内嵌的json
参考：
https://docs.fluentd.org/filter/parser
https://github.com/fluent/fluentd-kubernetes-daemonset/issues/181
https://carlos.mendible.com/2019/02/10/kubernetes-mount-file-pod-with-configmap/

定义一个ConfigMap用于映射Fluentd的配置文件

```bash
cat > config-map-parse-dotnet-log-field.yaml <<'EOF'
apiVersion: v1
kind: ConfigMap
metadata:
  name: fluentd-config-dotnet
  namespace: kube-system
data:
  parse-dotnet-log-field.conf: |
    <filter kubernetes.var.log.containers.*dotnet**>
      @type parser
      key_name log
      reserve_time true
      <parse>
        @type json
      </parse>
    </filter>
EOF

kubectl apply -f config-map-parse-dotnet-log-field.yaml
```

```bash
wget https://raw.githubusercontent.com/fluent/fluentd-kubernetes-daemonset/master/fluentd-daemonset-elasticsearch-rbac.yaml

# TIPS 如提示资源不可修改，尝试删除旧的daemonset
# kubectl  delete  daemonset fluentd --namespace kube-system

# --begin-- 微调，并存储为 fluentd-daemonset-elasticsearch-rbac-fixed.yaml
# 关闭TLS
#          - name: FLUENT_ELASTICSEARCH_SSL_VERIFY
#            value: "true"
# 调整ELASTICSEARCH_HOST
#          - name:  FLUENT_ELASTICSEARCH_HOST
#            value: "elasticsearch-master"
# 配置K8S_NODE_NAME名称，否则fluentd会报错
#          - name: K8S_NODE_NAME
#            valueFrom:
#              fieldRef:
#                fieldPath: spec.nodeName
# 调整dockercontainerlogdirectory volumes设置(注意，文件提供了两个开关，有两处控制，都需要调整)
#          我的实验场景属于 actual pod logs in /var/lib/docker/containers，如：
  # volumeMounts:
  # When actual pod logs in /var/lib/docker/containers, the following lines should be used.
  # - name: dockercontainerlogdirectory
  #   mountPath: /var/lib/docker/containers
  #   readOnly: true
  # volumes:
  # # When actual pod logs in /var/lib/docker/containers, the following lines should be used.
  # - name: dockercontainerlogdirectory
  #   hostPath:
  #     path: /var/lib/docker/containers
# 启用json解析
  # volumeMounts:
  # - name: fluentd-config-volume
  #   mountPath: /fluentd/etc/conf.d/parse-dotnet-log-field.conf
  #   subPath: parse-dotnet-log-field.conf
  # volumes:
  # - name: fluentd-config-volume
  #   configMap:
  #     name: fluentd-config-dotnet

# --end-- 微调，并存储为 fluentd-daemonset-elasticsearch-rbac-fixed.yaml

kubectl  apply -f fluentd-daemonset-elasticsearch-rbac-fixed.yaml

```


```
cd /fluentd/etc
mkdir conf.d
cat conf.d/parse-log-field.conf <<EOF
<filter kubernetes.var.log.containers.*dotnet*>
  @type parser
  key_name log
  reserve_time true
  <parse>
    @type json
  </parse>
</filter>
EOF
```

### 验证示例C#程序

#### 创建示例代码

```bash
mkdir dotnet-console-sample
cd dotnet-console-sample/

dotnet new console

dotnet add package Serilog.Sinks.Console 
dotnet add package Serilog.Enrichers.Context 
dotnet add package Serilog.Enrichers.EnrichedProperties 
dotnet add package Serilog.Enrichers.Thread

cat > Program.cs <<'EOF'
using System.Threading.Tasks;
using Serilog;

Log.Logger = new LoggerConfiguration()
 .Enrich.WithThreadId()
 .WriteTo.Console()
    .CreateLogger();

Log.Information("--begin--");

int counter=1;
do{
    Log.Information("counter: {counter}", counter);
    await Task.Delay(TimeSpan.FromSeconds(1));
    counter++;

}while(true);


Log.Information("--end--");

Log.CloseAndFlush();
EOF

cat > Dockerfile <<'EOF'
FROM mcr.microsoft.com/dotnet/runtime:6.0 AS base
WORKDIR /app

FROM mcr.microsoft.com/dotnet/sdk:6.0 AS build
WORKDIR /src
COPY ["dotnet-console-sample.csproj", "./"]
RUN dotnet restore "dotnet-console-sample.csproj"
COPY . .
WORKDIR "/src/"
RUN dotnet build "dotnet-console-sample.csproj" -c Release -o /app/build

FROM build AS publish
RUN dotnet publish "dotnet-console-sample.csproj" -c Release -o /app/publish

FROM base AS final
WORKDIR /app
COPY --from=publish /app/publish .
ENTRYPOINT ["dotnet", "dotnet-console-sample.dll"]

EOF

cat > .dockerignore <<'EOF'
**/.dockerignore
**/.env
**/.git
**/.gitignore
**/.project
**/.settings
**/.toolstarget
**/.vs
**/.vscode
**/.idea
**/*.*proj.user
**/*.dbmdl
**/*.jfm
**/azds.yaml
**/bin
**/charts
**/docker-compose*
**/Dockerfile*
**/node_modules
**/npm-debug.log
**/obj
**/secrets.dev.yaml
**/values.dev.yaml
LICENSE
README.md
EOF

```

#### 生成镜像

```bash
docker build -t dotnet-console-sample .
```

#### 测试镜像
```bash
docker run --rm -it dotnet-console-sample
```

#### 为MiniKube更新镜像
##### 方法一，将Docker镜像加载到为MiniKube的仓库中(我这边，这个方法不太靠谱，镜像总是不更新)

```bash
#https://minikube.sigs.k8s.io/docs/handbook/pushing/#2-push-images-using-cache-command
#minikube cache add dotnet-console-sample
#https://minikube.sigs.k8s.io/docs/handbook/pushing/#7-loading-directly-to-in-cluster-container-runtime
minikube image load dotnet-console-sample
#TODO cache reload?
```

##### 方法二，在Minikube的Docker中构建镜像(推荐)

```bash
eval $(minikube -p minikube docker-env)
# you will get something like:
# export DOCKER_TLS_VERIFY="1"
# export DOCKER_HOST="tcp://192.168.49.2:2376"
# export DOCKER_CERT_PATH="/home/congzhang/.minikube/certs"
# export MINIKUBE_ACTIVE_DOCKERD="minikube"
docker build -t dotnet-console-sample .
# To point your shell to minikube's docker-daemon, run:
# eval $(minikube -p minikube docker-env)

eval $(minikube docker-env -u)
```

#### 快速测试Pod

```bash
kubectl delete pods dotnet-console-sample ; kubectl apply -f dotnet-console-sample.yaml ; kubectl logs -f dotnet-console-sample

# 或者

kubectl delete pods dotnet-console-sample ; kubectl run -it --rm dotnet-console-sample --restart=Never --image  dotnet-console-sample --overrides='{ "apiVersion": "v1" , "spec": { "containers": [{"name":"dotnet-console-sample", "image":"dotnet-console-sample", "imagePullPolicy": "Never" }] } }'
```

#### 配置dotnet-console-sample Pod

Add dotnet-console-sample.yaml

```yaml
## dotnet-console-sample.yaml
apiVersion: v1
kind: Pod
metadata:
  name: dotnet-console-sample
spec:
  containers:
  - name: dotnet-console-sample
    image: dotnet-console-sample
    imagePullPolicy : Never    
```

#### 启动Pod

```bash
kubectl apply -f dotnet-console-sample.yaml
```

### 部署metricbeat, 跟踪系统指标(如CPU,内存,网络,磁盘,网络流量等)(可玩项)

```bash
helm install metricbeat elastic/metricbeat

# open you kibana now
# 1. add index pattern like Metricbeat-*
# 2. discovery something

```



# 参考链接

1. https://techcommunity.microsoft.com/t5/core-infrastructure-and-security/getting-started-with-logging-using-efk-on-kubernetes/ba-p/1333050
2. https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/#install-using-native-package-management
3. https://minikube.sigs.k8s.io/docs/start/
4. https://kubernetes.io/docs/tasks/tools/
5. https://helm.sh/docs/intro/install/
6. https://kubernetes.io/docs/concepts/cluster-administration/logging/
7. https://docs.fluentd.org/container-deployment/kubernetes
8. https://www.elastic.co/beats/metricbeat
9. https://kubernetes.io/docs/tutorials/stateless-application/expose-external-ip-address/
10. https://mherman.org/blog/logging-in-kubernetes-with-elasticsearch-Kibana-fluentd/
11. https://logz.io/blog/deploying-the-elk-stack-on-kubernetes-with-helm/
12. https://medium.com/kubernetes-tutorials/learn-how-to-assign-pods-to-nodes-in-kubernetes-using-nodeselector-and-affinity-features-e62c437f3cf8
13. https://minikube.sigs.k8s.io/docs/handbook/accessing/
14. https://kubernetes.io/docs/concepts/services-networking/service/
15. https://stackoverflow.com/questions/41509439/whats-the-difference-between-clusterip-nodeport-and-loadbalancer-service-types
17. https://github.com/elastic/helm-charts/tree/master/metricbeat
18. https://gist.githubusercontent.com/vineet68sharma/bdbd8a96f162ef119e9bc66bd47e6b8d/raw/f84a28535c204cd82aaf1eaf1d2265cd64d0d1e0/efk9.yaml
19. https://github.com/fluent/fluentd-kubernetes-daemonset/blob/master/fluentd-daemonset-elasticsearch-rbac.yaml
20. https://minikube.sigs.k8s.io/docs/handbook/pushing/
21. https://medium.com/bb-tutorials-and-thoughts/how-to-use-own-local-doker-images-with-minikube-2c1ed0b0968
22. https://stackoverflow.com/questions/42564058/how-to-use-local-docker-images-with-minikube
23. https://kubernetes.io/docs/concepts/containers/images/
24. https://minikube.sigs.k8s.io/docs/handbook/pushing/#7-loading-directly-to-in-cluster-container-runtime
25. https://medium.com/@alexdimango/elasticsearch-with-docker-in-five-minutes-401f0c5e403d
26. https://discuss.elastic.co/t/set-password-and-user-with-docker-compose/225075/12
27. https://www.elastic.co/guide/en/elasticsearch/reference/current/built-in-users.html
28. https://demo.elastic.co/app/apm/services?rangeFrom=now-15m&rangeTo=now
29. https://kubernetes.io/docs/tasks/inject-data-application/environment-variable-expose-pod-information/
30. https://docs.fluentd.org/filter/parser
31. https://github.com/fluent/fluentd-kubernetes-daemonset/issues/181
32. https://docs.fluentd.org/filter
33. https://carlos.mendible.com/2019/02/10/kubernetes-mount-file-pod-with-configmap/
34. https://kubernetes.io/docs/tasks/configure-pod-container/configure-pod-configmap/#create-configmaps-from-literal-values
35. https://stackoverflow.com/questions/45492166/reload-containerized-fluentd-configuration
36. https://stackoverflow.com/questions/46061672/elasticsearch-only-finding-hits-with-keyword-appended



